# HTTP请求核心原理

* 网络请求架构图：<img src='../images/javaee/http请求结构图.png' width='350px'>
* 负载均衡架构：LVM（传输层负载均衡），Nginx（应用层负载均衡），HAProxy（应用层负载均衡）
    * 硬件负载均衡（F5）性能，但是贵且不能动态扩容（配置的机器数是死的，可以用云ELB解决）
    * 软件负载均衡成本低，但是层次多会增加网络延迟
    * 数据链路层负载均衡：集群使用同样的IP，均衡器将IP映射到不同的MAC地址实现路由
    * 网络层负载均衡：集群使用不同的IP，均衡器直接将IP映射到不同的机器实现路由
    * 传输层负载均衡：通过IP+PORT映射实现路由
    * 应用层负载通过：通过请求URL反向代理实现路由，DNS负载均衡，将域名解析成不同的IP地址，虽然域名解析中的A记录表示一个IP对应一个域名，但是可以同时存在多个A记录，也就是可以存在一个域名对应多个IP，只是保存在多个A记录中；这种负载均衡优点就是速度快，缺点就是有缓存，一旦某台服务挂了由于缓存会导致大量访问不可达
    * 相关算法：
        * 轮训，随机：静态的负载算法，不利于写数据，每次机器是随机选的，状态丢失，一般应用在应用服务器集群或数据库只读集群
        * Hash，一致性Hash：根据请求方的IP，利用Hash取模求出目的IP，能保证读写问题，一般应用在缓存集群
        * 根据主键范围负载：每个机器存放不同的范围的主键范围，根据主键范围选择对应的机器（访问热点不均匀）
        * 对主键取模负载：根据主键取模结果选择不同的机器
        * 消息队列：将耗时的操作放入消费者队列，消费者自己从队列中拉取任务（延时问题）
        * 比率，随机加权：给每个服务器一个加权值（根据权值大小将这台机器在轮训中重复几次），
* DNS域名解析步骤：<img src='../images/javaee/dns域名解析步骤.png' width='350px'>
    * 从浏览器缓存中查找域名IP映射
    * 从操作系统中查找域名IP映射，Windows中的hosts文件，Linux中的named.conf
    * 从本地区的域名服务器中查找域名IP映射
    * 从根域名服务器中查找主域名服务器，解析com、cn、org等国际域名
    * 从主域名服务器中查找域名注册地服务器，也就是域名提供商的服务器地址
    * 从域名服务商的服务器中查找域名IP映射
    * 本地区域名服务器缓存这个新的域名IP映射
    * 本地区域名服务器将结果返回给客户端浏览器
    * 域名解析工具：windows和linux：nslookup，linux：dig
    * 清空域名缓存工具：windows：ipconfig /flushdns，linux：/etc/init.d/nscd restart
* CDN内容分布网络：<img src='../images/javaee/cdn解析步骤.png' width='350px'>
    * 将网站内容发布到最接近用户的网络“边缘”
    * CDN=镜像+缓存+整体负载均衡
    * 可扩展，安全性，可靠性、相应和执行
    * 首先需要解析静态文件域名对应的IP，同DNS解析步骤一致，唯一的区别就是内容的IP是存在公司的DNS服务器中的
    * 公司的DNS域名服务器会将静态文件所存储的CDN服务器IP返回
    * 返回的IP一般是CND服务器集群中的DNS负载均衡服务器
    * DNS负载均衡服务器会根据用户的区域，返回离用户最近的CDN服务器IP
    * 用户客户端直接去该IP服务器中拿去静态文件
* TCP状态转移图：<img src='../images/javaee/tcp状态转移.png' width='350px'>
    * CLOSED：起始点，超时和连接关闭时处于该状态
    * LISTEN：服务端等待连接的状态，对于有socket的bind，listen
    * SYN-SENT：客户端发起连接请求，发送SYN给服务端等待确认连接
    * SYN-RCVD：服务端接收到客户端的SYN请求，服务端有LISTEN转换为该状态，服务端回应客户端ACK+SYN；当客户端发送SYN的同时受到服务端的SYN标志，该客户端会进入到该状态
    * ESTABLISHED：服务端和客户端完成三次握手后进入的状态
    * FIN-WAIT-1：主动关闭的一方，第一次发送FIN给对方
    * FIN-WAIT-2：主动关闭的一方，接收到对方的ACK+FIN后，此时已经不能接收数据了，只能发送数据
    * CLOSE-WAIT：被动关闭的一方，接收到对方的FIN，并且发送ACK给对方
    * LAST-ACK：被动关闭的一方，发送FIN给对方，并在接收到ACK后进入CLOSED状态
    * CLOSING：两边同时发起关闭请求，操作都是接收到对方的FIN后发送一个ACK
    * TIME-WAIT：在FIN-WAIT-2时接收到对方的FIN标志时进入该状态；在CLOSING时双方同时收到FIN并发出ACK后进入该状态；在FIN-WAIT-1同时接收到了对方的ACK+FIN时进入该状态，其实就是跳过了FIN-WAIT-2状态
        * TIME-WAIT数量太多时，可能会是服务器开放的端口数太少，netstat查看网络统计信息
* HTTP请求在容器中的路由：<img src='../images/javaee/http请求在容器中的路由.png' width='450px'>

****

# HTTP网络请求相关基础概念

* Ctrl+F5强制不使用缓存，在请求头部中添加Pragma: no-cache和Cache-Control: no-cache两个字段
* TCP网络调优相关参数请参考《JavaWeb深入分析技术内幕》第73页
* 网络IO调优
    * 减少网络交互次数：客服端、服务端设置缓存，合并请求，合并JS、CSS等文件
    * 减小网络传输数据量的大小：压缩数据再传输
    * 尽量减少编码：尽量以字节的方式发送数据，在到双端进行处理
* HTTP通信过程中的编码解码：
    * 计算机最小存储结构字节无法表达所有自然语言，因此char到byte必须编码
    * UTF-8变长字节编码，如果单字节首位为0，则表示是ASCII字符；如果单字节首位是11开头，则表示是双字节的开头；如果单字节首位是10开头，则表示不是首字节，需要去上一字节寻找首字节
    * 需要编码的场景：IO操作，内存操作
    * UTF-8性能介于GBK和UTF-16之间，UTF-16是内存使用的编码，但是不适合网络传输，原始是网络传输会有丢包情况，UTF-16一旦丢失一字节数据就无法还原，UTF-8更加适合网络传输
    * 一次HTTP请求需要编码解码的部分，URL、Cookie、POST表单请求需要在客户端编码，在服务端解码，数据库读取的数据需要在服务端编码客户端解码
    * URL请求编码解码：
        * PathInfo是用UTF-8编码，QueryString是用GBK编码（在HTTP Header ContentType Charset指定，默认为ISO-8859-1）
        * PathInfo在服务端在解析URL时完成解码，是通过```<Connector URLEncoding='UTF-8'>```指定的编码集解码
        * QueryString是在服务端第一次调用```request.getParamter```进行解码，服务端设定```<Connector URLEncoding='UTF-8' useBodyEncodingForURI='true'>```指定使用HTTP Header ContentType Charset解码，```useBodyEncodingForURI='true'```只指定QueryString的解码
    * HTTP Header编码解码：在调用```request.getHeader```时发生，默认使用ISO-8859-1编解码，而且无法改变，所以在Header中需要传输非ASCII码字符，需要用URLEncoder编码，然后人为使用URLDecoder解码
    * POST表单编码解码：处理方式与QueryString一致，但是一定要在第一次调用```request.getParamter```时，调用```request.setCharacterEncoding(charset)```
    * HTTP Body编码解码：服务端编码集通过```response.setCharacterEncoding(charset)```设定，并会通过HTTP Header ContentType Charset返回给浏览器，浏览器首先会通过HTTP Header ContentType Charset进行解码，如果找不到则会找HTML中meta中指定的编码解码，否则就是默认ISO-8859-1解码
* Cookie和Session：
    * Cookie保存在浏览器端，通过httpheader进行传输，是一串保存了key，value键值对的字符串
    * Session是基于Cookie实现，在Cookie中存储了一个JSESIONOD的key值，避免Cookie太大，每次传输这个key值就行，服务端根据这个key能获取存储在服务端的key，value值；如果浏览器不支持Cookie则会将这个ID通过Path传给服务端，否则通过Cookie的JSESIONOD传给服务端

****

# Tomcat
* tomcat类加载器继承结构：<img src='../images/javaee/tomcat类加载器.png' width='250px'>
* web容器，包括tomcat、jetty、weblogic、websphere加载类时需要保证：
    * 部署在同一个服务器上的两个web应用程序所使用的java类库实现相互隔离
    * 部署在同一个服务器上的两个web应用程序所使用的java类库实现相互共享
    * 服务器尽可能保障自身的安全不受web应用服务器影响
    * 需要支持Hotswap功能
    * common目录：类库可供tomcat和web应用程序共同使用，CommonClassLoader
    * server目录：类库只供tomcat本身使用，web应用程序不可见，CatalinaClassLoader
    * shared目录：可供web应用程序使用，tomcat本身不可见，SharedClassLoader
    * webapp/web-inf：可被此web应用程序使用，对其他web应用程序和tomcat本身都不可见，WebappClassLoader
* tomcat加载本身类的过程：
    * 在Bootstrap类的initClassLoaders创建StandardClassLoader类加载器实例，并作为Bootstrap的catalinaLoader属性，作为整个tomcat的根类加载器
    * StandardClassLoader作为代理方法，仍然使用的是AppClassLoader类的方法，当且仅当tomcat的ClassPath没有设置时才会使用StandardClassLoader类加载器加载类，StandardClassLoader查找ClassPath是tomcat config.xml中的context元素中指定的ClassPath
* tomcat加载应用servlet过程：
    * 通用版：
        * tomcat通过StandardContext表示一个应用，并且通过StandardContext解析应用的web.xml加载应用中的servlet，每个servlet都是显示加载
        * StandardContext的startInternal方法会初始化tomcat的类加载器WebappClassLoader
        * 检查WebappClassLoader是否已经加载过该servlet类，如果是肯定在resourceEntries中有缓存
        * 其次到JVM中查找该sevlet是否被加载过，其实就是从JVM缓存中查找加载过的servlet类
        * 如果缓存没有中，就调用AppClassLoader加载该sevlet类，从JVM的ClassPath中查找该类
        * 如果该servlet没有被AppClassLoader加载，则会判断是否属于StandardClassLoader类加载器加载，如果应用时放在webapp下则成立
        * 最后会由WebappClassLoader在WEB-INF/classes目录下查找需要加载的servlet类，并添加到resourceEntries缓存中
    * 细节版：
        * tomcat会调用addwebapp方法会创建StandardContext，一个StandardContext代表一个web应用
        * StandardContext会创建XML解析对象，同时会解析context.xml、Host等相关配置文件，同时会启动ContextConfig解析其他配置
        * ContextConfig会获取相关资源，解析web.xml文件，将servlet、listener、filter等包装到ServletWrapper中，并且创建ClassLoader
        * ServletWrapper通过loadServlet加载servlet，并通过initSevlet初始化servlet，其实就是代理调用servlet自己的init方法
        * 与servlet相关类，ServletConfig、ServletRequest、ServletResponse以及ServletContext
* tomcat各大组件
    * Service，将Container和Connector封装，给外部提供服务；可以有多个Connector但是只能有一个Container
    * Server，提供接口，让其他程序可以访问Service提供的服务，以及管理Service的生命周期和寻找服务的Service
    * Connector，负责接收浏览器发过来的TCP请求，并且解析TCP相关传输数据，将传输数据封装到Request和Response对象中，并开辟一个线程将Request和Response对象传送给Container处理该请求
    * Container，容器的父接口，所有子容器必须实现这个接口，采用责任链方式处理请求。有四个子容器，Engine、Host、Context、Wrapper
        * Engine主要负责管理容器，是最上层的容器
        * Host代表Engine中的虚拟主机，主要用来运行应用，安装、展开这些应用
        * Context包含Servlet所需要的上下文，并且负责定位Servlet
        * Wrapper就是ServletWrapper，主要用来管理Servlet，包括Servlet加载、初始化、执行和资源回收
    * 其他组件：security、logger、session、mbeans、naming

****

# ISO七层协议
* OSI七层模型通过七个层次化的结构模型使不同的系统不同的网络之间实现可靠的通讯，因此其最主要的功能就是帮助不同类型的主机实现数据传输
    <table>
        <tr>
            <td><img src='../images/javaee/osi七层网络协议.gif' width='450px'></td>
            <td><img src='../images/javaee/osi七层网络协议示例.jpg' width='450px'></td>
        </tr>
    </table>
* 一个设备工作在哪一层，关键看它工作时利用哪一层的数据头部信息。网桥工作时，是以MAC头部来决定转发端口的，因此显然它是数据链路层的设备   
    * 物理层：网卡，网线，集线器，中继器，调制解调器
    * 数据链路层：网桥，交换机
    * 网络层：路由器
    * 网关工作在第四层传输层及其以上
* 交换机和路由器的区别
    * 交换机
        * 交换机是一种基于MAC地址识别，能完成封装转发数据包功能的网络设备。交换机可以“学习”MAC地址，并把其存放在内部地址表中，通过在数据帧的始发者和目标接收者之间建立临时的交换路径，使数据帧直接由源地址到达目的地址
    * 路由器
        * 路由器的一个作用是连通不同的网络，另一个作用是选择信息传送的线路。选择通畅快捷的近路，能大大提高通信速度，减轻网络系统通信负荷，节约网络系统资源，提高网络系统畅通率
    * 路由器与交换机的主要区别
        * 工作层次不同：最初的的交换机是工作在数据链路层，而路由器一开始就设计工作在网络层。由于交换机工作在数据链路层，所以它的工作原理比较简单，而路由器工作在网络层，可以得到更多的协议信息，路由器可以做出更加智能的转发决策
        * 数据转发所依据的对象不同：交换机是利用物理地址或者说MAC地址来确定转发数据的目的地址。而路由器则是利用IP地址来确定数据转发的地址。IP地址是在软件中实现的，描述的是设备所在的网络。MAC地址通常是硬件自带的，由网卡生产商来分配的，而且已经固化到了网卡中去，一般来说是不可更改的。而IP地址则通常由网络管理员或系统自动分配
        * 传统的交换机只能分割冲突域，不能分割广播域；而路由器可以分割广播域：由交换机连接的网段仍属于同一个广播域，广播数据包会在交换机连接的所有网段上传播，在某些情况下会导致通信拥挤和安全漏洞。连接到路由器上的网段会被分配成不同的广播域，广播数据不会穿过路由器。虽然第三层以上交换机具有VLAN功能，也可以分割广播域，但是各子广播域之间是不能通信交流的，它们之间的交流仍然需要路由器
        * 路由器提供了防火墙的服务：路由器仅仅转发特定地址的数据包，不传送不支持路由协议的数据包传送和未知目标网络数据包的传送，从而可以防止广播风暴
* 网络分层介绍
    * 物理层：在OSI参考模型中，物理层（Physical Layer）是参考模型的最低层，也是OSI模型的第一层
        * 物理层的主要功能是：利用传输介质为数据链路层提供物理连接，实现比特流的透明传输
    * 数据链路层：数据链路层（Data Link Layer）是OSI模型的第二层，负责建立和管理节点间的链路
        * 该层的主要功能是：通过各种控制协议，将有差错的物理信道变为无差错的、能可靠传输数据帧的数据链路，检查验证数据包的正确性
        * 该层通常又被分为介质访问控制（MAC）和逻辑链路控制（LLC）两个子层
            * MAC子层的主要任务是解决共享型网络中多用户对信道竞争的问题，完成网络介质的访问控制
            * LLC子层的主要任务是建立和维护网络连接，执行差错校验、流量控制和链路控制
    * 网络层（Network Layer）是OSI模型的第三层，是通信子网的最高一层
        * 该层的主要功能是：通过路由选择算法，为报文或分组通过通信子网选择最适当的路径。将数据根据网络地址IP对数据拆分，并发送到不同的网络
        * 在实现网络层功能时，需要解决的主要问题如下：
            * 寻址：在不同子网之间通信时，为了识别和找到网络中的设备，每一子网中的设备都会被分配一个唯一的地址。由于各子网使用的物理技术可能不同，因此这个地址应当是逻辑地址（如IP地址）
            * 交换：规定不同的信息交换方式。常见的交换技术有：线路交换技术和存储转发技术，后者又包括报文交换技术和分组交换技术
            * 路由算法：当源节点和目的节点之间存在多条路径时，本层可以根据路由算法，通过网络为数据分组选择最佳路径，并将信息从最合适的路径由发送端传送到接收端
            * 连接服务：与数据链路层流量控制不同的是，前者控制的是网络相邻节点间的流量，后者控制的是从源节点到目的节点间的流量。其目的在于防止阻塞，并进行差错检测
    * 传输层，OSI下3层的主要任务是数据通信，上3层的任务是数据处理。而传输层（Transport Layer）是OSI模型的第4层。因此该层是通信子网和资源子网的接口和桥梁，起到承上启下的作用
        * 该层的主要任务是：向用户提供可靠的端到端的差错和流量控制，保证报文的正确传输。该层常见的协议：TCP/IP中的TCP协议、Novell网络中的SPX协议和微软的NetBIOS/NetBEUI协议，将数据运送到相关的网络，决定数据所属的网络
        * 传输层提供会话层和网络层之间的传输服务，这种服务从会话层获得数据，并在必要时，对数据进行分割。传输层的主要功能如下：
            * 传输连接管理：提供建立、维护和拆除传输连接的功能。传输层在网络层的基础上为高层提供“面向连接”和“面向无接连”的两种服务
            * 处理传输差错：提供可靠的“面向连接”和不太可靠的“面向无连接”的数据传输服务、差错控制和流量控制。在提供“面向连接”服务时，通过这一层传输的数据将由目标设备确认，如果在指定的时间内未收到确认信息，数据将被重发
            * 监控服务质量
    * 会话层（Session Layer）是OSI模型的第5层，是用户应用程序和网络之间的接口
        * 该层的主要功能是：向两个实体的表示层提供建立和使用连接的方法，将不同实体之间的表示层的连接称为会话。域名解析解释数据将发到哪个网络
        * 用户可以按照半双工、单工和全双工的方式建立会话。会话层的具体功能如下：
            * 会话管理：允许用户在两个实体设备之间建立、维持和终止会话，并支持它们之间的数据交换。例如提供单方向会话或双向同时会话，并管理会话中的发送顺序，以及会话所占用时间的长短
            * 会话流量控制：提供会话流量控制和交叉会话功能
            * 寻址：使用远程地址建立会话连接
            * 出错控制：从逻辑上讲会话层主要负责数据交换的建立、保持和终止，但实际的工作却是接收来自传输层的数据，并负责纠正错误。会话控制和远程过程调用均属于这一层的功能。但应注意，此层检查的错误不是通信介质的错误，而是磁盘空间、打印机缺纸等类型的高级错误
    * 表示层（Presentation Layer）是OSI模型的第六层，它对来自应用层的命令和数据进行解释，对各种语法赋予相应的含义
        * 该层的主要功能是：“处理用户信息的表示问题，如编码、数据格式转换和加密解密”等。表示层的具体功能如下：
            * 数据格式处理：协商和建立数据交换的格式，解决各应用程序之间在数据格式表示上的差异
            * 数据的编码：处理字符集和数字的转换。例如由于用户程序中的数据类型（整型或实型、有符号或无符号等）、用户标识等都可以有不同的表示方式，因此，在设备之间需要具有在不同字符集或格式之间转换的功能
            * 压缩和解压缩：为了减少数据的传输量，这一层还负责数据的压缩与恢复
            * 数据的加密和解密：可以提高网络的安全性
    * 应用层（Application Layer）是OSI参考模型的最高层，它是计算机用户，以及各种应用程序和网络之间的接口
        * 该层的主要功能是：为应用程序提高网络编程服务
        * 应用层为用户提供的服务和协议有：文件服务、目录服务、文件传输服务（FTP）、远程登录服务（Telnet）、电子邮件服务（E-mail）、打印服务、安全服务、网络管理服务、数据库服务等，应用层的主要功能如下：
            * 用户接口：应用层是用户与网络，以及应用程序与网络间的直接接口，使得用户能够与网络进行交互式联系。
            * 实现各种服务：该层具有的各种应用程序可以完成和实现用户请求的各种服务。
* TCP/IP分层模型<img src='../images/javaee/tcpip四层网络协议.png' width='350px'>
    * TCP/IP协议被组织成四个概念层，其中有三层对应于ISO参考模型中的相应层。TCP/IP协议族并不包含物理层和数据链路层，因此它不能独立完成整个计算机网络系统的功能，必须与许多其他的协议协同工作
    * 第一层：网络接口层，包括用于协作IP数据在已有网络介质上传输的协议。实际上TCP/IP标准并不定义与ISO数据链路层和物理层相对应的功能。相反，它定义像地址解析协议(Address Resolution Protocol,ARP)这样的协议，提供TCP/IP协议的数据结构和实际物理硬件之间的接口
    * 第二层：网间层，对应于OSI七层参考模型的网络层。本层包含IP协议、RIP协议(Routing Information Protocol，路由信息协议)，负责数据的包装、寻址和路由。同时还包含网间控制报文协议(Internet Control Message Protocol,ICMP)用来提供网络诊断信息
    * 第三层：传输层，对应于OSI七层参考模型的传输层，它提供两种端到端的通信服务。其中TCP协议(Transmission Control Protocol)提供可靠的数据流运输服务，UDP协议(Use Datagram Protocol)提供不可靠的用户数据报服务
    * 第四层：应用层，对应于OSI七层参考模型的应用层和表达层。因特网的应用层协议包括Finger、Whois、FTP(文件传输协议)、Gopher、HTTP(超文本传输协议)、Telent(远程终端协议)、SMTP(简单邮件传送协议)、IRC(因特网中继会话)、NNTP（网络新闻传输协议）等